In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import pandas as pd
import seaborn as sns
from IPython.display import display
import os, sys, itertools, csv
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.gene import get_coding_genetic_target_len_d, get_intergenic_len_d
from util.genome import get_feature_hit_set
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD
pd.options.display.max_columns = 100

In [2]:
assoc_muts_df = pd.read_pickle("./data/6_df.pkl")
display(assoc_muts_df.shape, assoc_muts_df.head())

(114, 45)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,pathways,pathway links,COGs,COG links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
2527,28,None,[crl],DEL,257908,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(257908, 258683)","{ECK120001080, ECK125229291, ECK125229290}","[{'name': 'crl', 'RegulonDB ID': 'ECK120001080...",False,False,{},{},{},{},{},True,"[{'name': 'insA9', 'RegulonDB ID': 'ECK1252292...","{'ECK120001080': ['ECK120001080'], 'ECK1252292...",[],{},[],{},[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2528,3,coding (163/210 nt),cspC,INS,1907273,NC_000913,(CGTCCTG)1→2,1,TOL_putrescine,50,1,1.0,1,True,"(1907273, 1907273)",{ECK120002092},"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",False,False,{},{},{},{},{},True,"[{'name': 'cspC', 'RegulonDB ID': 'ECK12000209...",{'ECK120002092': ['ECK120002092']},"[{'name': 'yobF-cspC', 'RegulonDB ID': 'ECK120...",{'ECK120028784': ['ECK120002092']},[],{},[],{},[{'name': 'Transcription'}],{'Transcription': ['ECK120028784']},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2529,29,None,"insB1, insA",DEL,1978503,NC_000913,Δ776 bp,1,TOL_putrescine,50,1,1.0,1,True,"(1978503, 1979278)","{ECK120023915, ECK120003616, ECK120003617}","[{'name': 'insAB-5', 'RegulonDB ID': 'ECK12002...",False,False,"{ECK120017150, ECK120013472}","{ECK120034252, ECK125095446}",{},{},{},True,"[{'name': 'flhDC TFBS', 'RegulonDB ID': 'ECK12...","{'ECK120023915': ['ECK125095446', 'ECK12002391...","[{'name': 'flhDC', 'RegulonDB ID': 'ECK1200149...","{'ECK120014934': ['ECK120023915', 'ECK12002391...","[{'name': 'OmpR', 'significantly associated co...","{'MatA': ['flhDC', 'flhDC', 'flhDC'], 'OmpR': ...",[],{},"[{'name': 'Mobilome: prophages, transposons'},...","{'Mobilome: prophages, transposons': ['ECK1250...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2530,32,intergenic (‑1/+1),gatC/gatC,DEL,2173363,NC_000913,Δ2 bp,1,TOL_putrescine,50,1,1.0,1,False,"(2173363, 2173364)",{ECK120002276},"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",False,False,{},{},{},{},{},False,"[{'name': 'gatC', 'RegulonDB ID': 'ECK12000227...",{'ECK120002276': ['ECK120002276']},"[{'name': 'gatYZABCD', 'RegulonDB ID': 'ECK120...",{'ECK120015849': ['ECK120002276']},"[{'name': 'ArcA', 'significantly associated co...","{'ArcA': ['gatYZABCD'], 'GatR': ['gatYZABCD'],...",[],{},[],{},37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
2531,4,N34K (AAT→AAG),mreB,SNP,3400986,NC_000913,A→C,1,TOL_putrescine,50,1,1.0,1,True,"(3400986, 3400986)",{ECK120000601},"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",False,False,{},{},{},{},{},True,"[{'name': 'mreB', 'RegulonDB ID': 'ECK12000060...",{'ECK120000601': ['ECK120000601']},"[{'name': 'mreBCD', 'RegulonDB ID': 'ECK120030...",{'ECK120030764': ['ECK120000601']},[],{},[],{},"[{'name': 'Cell cycle control, cell division, ...","{'Cell cycle control, cell division, chromosom...",37 celsius,glucose(2),putrescine,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)


# Build genetic feature links dictionary for mutflow

In [3]:
# # Now in NB 2.2.1. Remove cell becuause obsolete
# def get_genetic_links_d(mut_row):
#     gen_trgt_d = dict()
#     feat_df = pd.DataFrame()
#     for feat_d in mut_row["genomic features"]:
#         feat_df = feat_df.append(feat_d, ignore_index=True)
#     for gen_trgt in mut_row["genetic features"]:
#         gen_trgt_d[gen_trgt["RegulonDB ID"]] = list(get_feature_hit_set(gen_trgt["range"], feat_df, "range", "RegulonDB ID"))
#     return gen_trgt_d


# assoc_muts_df["genetic feature links"] = assoc_muts_df.apply(get_genetic_links_d, axis=1)
# assoc_muts_df.head()

# Significance study (level specific)

In [4]:
from util.feature import get_feat_mut_cnt_df_from_links
comp_mut_df = get_feat_mut_cnt_df_from_links(assoc_muts_df, "genetic features", "genetic feature links")
comp_mut_df

,length,observed mutation count,name
ECK120001080,1178,7,crl
ECK125229290,504,7,insB9
ECK125229291,276,7,insA9
ECK120002092,210,2,cspC
ECK120023915,698,21,insAB-5
...,...,...,...
ECK120000629,1221,2,nagC
ECK120003633/ECK120003634,197,1,dsrA/yedP
ECK120000764,993,1,proX
ECK120000064,1344,1,argG


In [5]:
for _, r in assoc_muts_df.iterrows():
    for feat_d in r["genetic features"]:
        feat_d["mutation set count"] = comp_mut_df.loc[feat_d["RegulonDB ID"]]["observed mutation count"]

In [6]:
mutated_seq_len = np.sum(comp_mut_df["length"])

TOTAL_GENOME_LEN = 4641652
genome_remainder_len = TOTAL_GENOME_LEN - mutated_seq_len
df = pd.DataFrame([[0, genome_remainder_len]], columns=["observed mutation count", "length"], index=["genome remainder"])
comp_mut_df = comp_mut_df.append(df, sort=False)
total_seq_len = TOTAL_GENOME_LEN


comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
comp_mut_df.head()

,length,observed mutation count,name,proportion
ECK120001080,1178,7,crl,0.000254
ECK125229290,504,7,insB9,0.000109
ECK125229291,276,7,insA9,0.000059
ECK120002092,210,2,cspC,0.000045
ECK120023915,698,21,insAB-5,0.000150


In [7]:
from collections import Counter
comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.
num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=num_muts,
        replace=True,
        p=proportion_l
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

display(comp_mut_df.head())

,length,observed mutation count,name,proportion,equal or larger count,p value,significant
ECK120001080,1178,7,crl,0.000254,0,0.0,True
ECK125229290,504,7,insB9,0.000109,0,0.0,True
ECK125229291,276,7,insA9,0.000059,0,0.0,True
ECK120002092,210,2,cspC,0.000045,0,0.0,True
ECK120023915,698,21,insAB-5,0.000150,0,0.0,True


In [8]:
from statsmodels.stats import multitest
pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=ENRICH_ALPHA,
    method=MULTI_HYP_CORR_METHOD)

pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

,length,observed mutation count,name,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
ECK120001080,1178,7,crl,0.000254,0,0.0,True,0.0,True
ECK125229290,504,7,insB9,0.000109,0,0.0,True,0.0,True
ECK125229291,276,7,insA9,0.000059,0,0.0,True,0.0,True
ECK120002092,210,2,cspC,0.000045,0,0.0,True,0.0,True
ECK120023915,698,21,insAB-5,0.000150,0,0.0,True,0.0,True


In [9]:
# # not wanting to consider mutation events with only 1 features as being significant
comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
# comp_mut_df.head()

In [10]:
for i, r in assoc_muts_df.iterrows():
    for d in r["genetic features"]:
        if d["RegulonDB ID"] in comp_mut_df.index:
            d["significantly enriched"] = comp_mut_df.loc[d["RegulonDB ID"]]["final significance"]

# Outputting dataframe for the next step in the pipeline

In [11]:
assert(len(assoc_muts_df.exp.unique())==1)
exp = assoc_muts_df.exp.unique()[0]
comp_mut_df.to_csv("./data/supp/permutation_test_results_per_experiment/"+exp+"_mut_gen_feat_signif.csv")

In [12]:
assoc_muts_df.to_pickle("./data/7_df.pkl")

In [13]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_ID(RegulonDB_ID, json):
#     name = ""
#     is_signif = False
#     for d in json:
#         if d["RegulonDB ID"] == RegulonDB_ID:
#             name = d["name"]
#             is_signif = d["significant"]
#             break
#     return (name, is_signif)


# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in assoc_muts_df.iterrows():
#     link_d = r["genetic feature links"]
#     for targ_ID, hits in link_d.items():
#         t, is_signif = _get_name_signif_from_ID(targ_ID, r["genetic features"])
#         t = str(t)
#         if is_signif:
#             t = '*' + t
#         if t not in targ_cnt_d.keys():
#             targ_cnt_d[t] = 0
#         targ_cnt_d[t] += len(hits)
        
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0",
#                   width=1
#                  )
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.2, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 genetic mutated feature count")
# ax.get_legend().remove()